In [2]:
with open("example_md_to_text.txt", "r", encoding="latin-1") as f: # TO DO: check proper encoding for .md files
    markdown_example = f.read()

with open("example.pdf", "rb") as f:
    pdf = f.read()

with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

with open("GROQ_KEY.txt", "r") as f:
    groq_token = f.read()

In [3]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

alberto-lorente


In [4]:
import torch

device = "cpu"
if torch.cuda.is_available():
  print("Cuda available")
  device = torch.device('cuda')

Cuda available


In [5]:
from council_rag.preprocessing import preprocess_markdown_text

In [6]:
paragraphs, clusters = preprocess_markdown_text(markdown_example,
                                                    model_id ="HIT-TMG/KaLM-embedding-multilingual-mini-v1", 
                                                    spacy_model="fr_core_news_sm", 
                                                    n_sents_per_para=10)

Cuda available
Number of clusters:  3
Score:  0.44600135

Number of clusters:  4
Score:  0.44329956

Number of clusters:  5
Score:  0.47113436

Number of clusters:  6
Score:  0.48168555

Number of clusters:  7
Score:  0.48469815

Number of clusters:  8
Score:  0.5539949

Index 5
Optimal Number of Clusters 8


In [7]:
paragraphs[0].keys()

dict_keys(['paragraph_union', 'start_range', 'stop_range', 'list_sents', 'para_embedding', 'para_cluster'])

In [8]:
paragraphs[0]

{'paragraph_union': '# Séance publique\n\n\n\n # du jeudi 28 octobre 2021\n\n\n\n # à 18h00\n\n\n\n# Chorum Alain Gilles - Halle Vacheresse\n\n\n\n # Rue des Vernes à Roanne\n\n\n\n #',
 'start_range': np.int64(0),
 'stop_range': np.int64(10),
 'list_sents': ['#',
  'Séance publique\n\n\n\n',
  '# du jeudi 28 octobre 2021\n\n\n\n',
  '# à 18h00\n\n\n\n#',
  'Chorum',
  'Alain Gilles',
  '- Halle Vacheresse\n\n\n\n',
  '#',
  'Rue des Vernes à Roanne\n\n\n\n',
  '#'],
 'para_embedding': array([-2.56996620e-02,  2.94211768e-02,  2.33180169e-02,  7.82801863e-03,
        -1.48487631e-02,  3.78930159e-02, -2.68976931e-02,  4.26564254e-02,
         5.15989289e-02, -7.38588581e-03, -4.01267707e-02, -1.80062763e-02,
         2.82343999e-02, -3.16207297e-02,  1.73622519e-02,  1.40902838e-02,
        -3.54822762e-02,  3.80511843e-02,  5.73955253e-02,  2.10540593e-02,
         1.22920386e-02,  2.19017286e-02,  7.41365105e-02,  5.48744798e-02,
         9.39616188e-03, -3.25019620e-02,  3.81131172e

In [9]:
clusters.keys()

dict_keys(['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'])

In [10]:
clusters["cluster_0"].keys()

dict_keys(['para_indexes', 'union_paras'])

In [11]:
clusters["cluster_0"]

{'para_indexes': [1,
  10,
  14,
  19,
  30,
  31,
  33,
  37,
  40,
  46,
  52,
  57,
  61,
  64,
  69,
  72,
  73,
  77,
  80,
  84,
  85,
  86,
  89],
 'union_paras': "PROCES VERBAL\n\n\n\n L'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle Vacheresse \x96 Rue des Vernes à Roanne.\n\n\n\n La convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n # Etaient présents :\n\n\n\nChristine Aranéo - Marcel AugierDana Burnichon| | |Edmond Bourgeon| |\n\n|-| | |Catherine Brun| |\n\n|Jean-Marc Detour| | | | |\n\n|Daniel Fréchet| | |Lucien Murzi| |\n\n|Quentin Guillermin|X| | | |\n\n|Jean-Paul Heyberger| | |Christelle Lattat| |\n\n|Franck Maupetit| | |Annie Gerenton| |\n\n|Patrick Meunier|X| | | |\n\n|Nabih Nejjar|X| | |(arrivé en cours de séance)|\n\n|Gilles Passot| | |Fabien Lambert|(arrivé

Takes around 25 secs to run in my machine.

## Data Transformations

### Summarization algos

The process is fairly straightforward, we take the union of the paragraphs in each cluster and then we generate a summary for each cluster using gemma2-9b-it in groq. Given how slow the hf models we tried where, we will just use groq.


In [12]:
from council_rag.data_transformations.text_transformations import generate_groq_summary

In [13]:
summary_prompt = """Il s'agit d'un texte concernant un projet de géothermie :
TEXTE: 
{}

TÂCHE:
Make a one page summary paying special attention to all the administrative matters like budgets, plans, actions to take in the future, organizational and hierarchical charts, announcements, meetings, contacts, elections, reports and all the related topics to these.

OUTPUT:
Publier directement le résumé.
"""

In [17]:
cluster_paras = []
for cluster in clusters.values():
    cluster_para = cluster["union_paras"]
    cluster_paras.append(cluster_para)

print(len(cluster_paras))

8


gemma2-9b-it is the model with the largest Token limit per minute, 15000.

llama-3.3-70b-versatile allows 6000 per minute.

In [ ]:
import time
# max tokens for llama-3.3-70b-versatile is 6000 per minute
cluster_summaries = []

token_limit = 14000
token_count = 0
for cluster_para in cluster_paras:
    # print(len(cluster_para))
    naive_length_check = len(cluster_para.split(" ")) # the token calcs in groq are very close to this number
    token_count += naive_length_check
    # print(naive_length_check)
    if token_count > token_limit:
        time.sleep(60)
        token_count = 0
    cluster_summary = generate_groq_summary(summary_prompt, cluster_para, groq_token, model="gemma2-9b-it")
    cluster_summaries.append(cluster_summary)
    
# from 6 minutes to 28 secs in my machine

In [31]:
import json
with open("cluster_summaries_example.json", "w") as f:
    json.dump(cluster_summaries, f)


## Table content algos

The process is a bit more complex. Since we want to put the augmented table context in the right place and with im2pdf we get the page where the table is, we have to estimate where in the running text the table is. A workaround could be extracting each page text of the pdf as markdown to keep track of what span of text happens in each page. Then we could just try to regex match the table context to the pages text and append the augmented table context above/below the table.

Note: some pdfs may have a page n/m at the end of the page but it is not always the case.


In [56]:
from council_rag.data_utils import pf2mdpagebypage, pdf2md


In [53]:
output_folder = "pdfs/"
input_pdf = "example.pdf"

In [ ]:
# using our fx with fitz and llama parse
pages = pf2mdpagebypage(input_pdf=input_pdf, output_folder=output_folder)

# takes around 30 secs per page to run in my machine
# fifteen minutes for 20 pages
# 30 for the whole pdf

Started parsing the file under job_id e6c3be36-3b22-4c6f-a210-074b96ce3634
Started parsing the file under job_id ee3e530b-af77-4a45-af4b-1e21fc5c9721
..Started parsing the file under job_id 6817bf31-447b-4700-8725-f951bc060787
.Started parsing the file under job_id c8279860-22ea-49d6-8c26-eeceedcc2258
Started parsing the file under job_id 713a79d8-92a4-4e3f-aa92-dcce4cb73b56
Started parsing the file under job_id 35ca6f2c-6f9b-48fd-91c0-ad6285ed1fd1
Started parsing the file under job_id 2c74e8b3-7165-4797-aa23-7088a5930839
..Started parsing the file under job_id f8f06f75-18dd-44ea-8e63-4a33309666ed
.Started parsing the file under job_id 2b7159d8-9efb-4b7b-979c-fce21aef6c6a
Started parsing the file under job_id 4d706a17-5246-41f4-b1a7-421ef3bec0c9
Started parsing the file under job_id 35705776-5d03-4651-a335-0a830d3c19e5
Started parsing the file under job_id 90e34f19-364a-49e0-8db0-d2f5b17e6d86
Started parsing the file under job_id 8abd3a47-d709-4609-a73b-6035b4db7a98
Started parsing the

In [55]:
pages[0]

"# Séance publique\n\n# du jeudi 28 octobre 2021 à 18h00\n\n# Chorum Alain Gilles - Halle Vacheresse\n\n# Rue des Vernes à Roanne\n\n# PROCES VERBAL\n\nL'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l’Espace Chorum – Halle Vacheresse – Rue des Vernes à Roanne.\n\nLa convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n# Etaient présents :\n\nChristine Aranéo - Marcel Augier - Jean-Jacques Banchet - Pierre Barnet - Martine Barroso - Isabelle Berthelot -\nFranck Beysson - Jean-Yves Boire - Romain Bost - Michelle Bouchet - Edmond Bourgeon - Marie-Christine Bravo -\nCatherine Brun - Dominique Bruyère - Nicolas Chargueros - Jean-Luc Chervin - Christine Chevillard - Pierre Coissard -\nPatrick Collet - Sandra Creuzet - Hervé Daval - Jean-Paul Descombes - Pierre Devedeux - Christian Dorange -\nDavid Dozance - Catherine 